1. Selenium et un browser headless

Objectif : comprendre ce qu'est Selenium et comment executer un navigateur en mode headless.

Selenium est un outil d'automatisation de navigateur. Il controle un vrai navigateur (Chrome, Firefox) via un driver.
Le mode headless permet d'executer le navigateur sans interface graphique.

Prerequis :
- Un navigateur installe (Chrome ou Firefox).
- Selenium installe (pip install selenium).

Note : Selenium 4 utilise Selenium Manager pour telecharger le driver automatiquement.

In [ ]:
# Import de base Selenium.
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions

# Construit les options Chrome.
options = ChromeOptions()
# Active le mode headless (nouvelle option Chrome).
options.add_argument('--headless=new')
# Desactive le GPU pour stabilite dans certains environnements.
options.add_argument('--disable-gpu')
# Fixe une taille de fenetre virtuelle.
options.add_argument('--window-size=1280,800')

# Construit le driver (Selenium Manager choisit le bon driver si Chrome est installe).
# Ne lancez pas ce bloc si vous n'avez pas de navigateur installe.
# driver = webdriver.Chrome(options=options)
# driver.quit()

2. Utiliser Selenium a la main

Objectif : piloter un navigateur pour charger une page, lire des elements et faire un screenshot.

Etapes :
- Lancer le driver avec options headless.
- Naviguer vers une URL.
- Extraire un element par CSS.
- Fermer le navigateur proprement.

In [ ]:
from selenium.webdriver.common.by import By

# URL de demonstration.
url = 'https://example.com/'

# Demarre le navigateur headless.
driver = webdriver.Chrome(options=options)
try:
    # Ouvre la page.
    driver.get(url)
    # Lit le titre de la page.
    title = driver.title
    print('Titre:', title)

    # Selectionne un element par CSS.
    heading = driver.find_element(By.CSS_SELECTOR, 'h1')
    print('H1:', heading.text)

    # Enregistre un screenshot local.
    driver.save_screenshot('example.png')
finally:
    # Ferme toujours le navigateur.
    driver.quit()

3. Utiliser Scrapy et Selenium ensemble

Objectif : combiner Scrapy (orchestration et pipelines) et Selenium (rendu JS).

Approche courante :
- Scrapy gere la planification des URLs.
- Selenium rend la page (JavaScript) et retourne le HTML.
- Scrapy parse ensuite ce HTML.

Le code ci-dessous montre une Spider qui utilise Selenium pour obtenir le HTML rendu.
Ce n'est pas la methode la plus performante, mais elle est simple a comprendre.

In [ ]:
import scrapy
from scrapy.http import HtmlResponse

class SeleniumSpider(scrapy.Spider):
    name = 'selenium_spider'
    start_urls = ['https://example.com/']

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Initialise le navigateur une seule fois.
        self.driver = webdriver.Chrome(options=options)

    def closed(self, reason):
        # Ferme le navigateur a la fin du crawl.
        if self.driver:
            self.driver.quit()

    def parse(self, response):
        # Utilise Selenium pour rendre la page.
        self.driver.get(response.url)
        # Recupere le HTML rendu par le navigateur.
        html = self.driver.page_source
        # Cree une nouvelle Response Scrapy a partir du HTML.
        rendered = HtmlResponse(url=response.url, body=html, encoding='utf-8')

        # Extrait des donnees avec Scrapy.
        title = rendered.css('title::text').get()
        yield {'url': response.url, 'title': (title or '').strip()}

4. Exercices

1. Ajouter un wait explicite pour attendre un element (WebDriverWait).
2. Extraire plusieurs elements et les mettre en liste.
3. Prendre un screenshot par URL et nommer le fichier.
4. Integrez Selenium dans un pipeline Scrapy (nettoyage + export JSON).
5. Essayer un site de demo avec JavaScript et comparer le HTML avant/apres rendu.
